In [1]:
import pandas as pd
import os
import ast
import pyflakes

# Constants
QUESTION_TYPE = 1
ANSWER_TYPE = 2

file = os.path.join('..', 'data', 'pandas-normalizedcode-dataset.csv')

dataset = pd.read_csv(file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")
print('Amount of entries in the dataset = %d' % len(dataset))
dataset


Amount of entries in the dataset = 3205


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,AnswerCount,CommentCount,Id,ParentId,PostTypeId,Score,Tags,Title,Body,ViewCount,Code,AllCode,ProcessedCode,PandasImportCode,PandasImportNormalized,DataFrameCode,DataFrameNormalized
0,0,0,0,2.0,7,7776679,NaN,1,25,<python><pandas>,append two data frame with pandas,<p>When I try to merge two dataframes by rows ...,25479.0,"['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', '', '', '']","['bigdata = data1.append(data2)\n', '', '', '']",False,['import pandas as pd\ndata2 = pd.DataFrame()\...,True
1,1,1,1,7.0,3,7837722,NaN,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,<p>I want to perform my own complex operations...,195364.0,"['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['2011-10-19,27.37,27.47,27.01,27.13,42880000,...","['2011-10-19,27.37,27.47,27.01,27.13,42880000,...",False,"['import pandas as pd\n2011-10-19,27.37,27.47,...",False
2,2,2,2,2.0,0,8916302,NaN,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,<p>I have a dataframe <code>df</code> in panda...,19805.0,"['df_greater_than10 = df[df[""colA""] > 10]\n']","['df', 'pandas.read_table', 'colA', 'df_greate...","['df_greater_than10 = df[df[""colA""] > 10]\n']","['df_greater_than10 = df[df[""colA""] > 10]\n']",False,['import pandas as pd\ndf = pd.DataFrame()\ndf...,True
3,3,3,3,3.0,16,8991709,NaN,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,"<p>I recently came across the <a href=""http://...",16173.0,[],"['pandas', 'data.table', 'data.table', 'merge(...",[],[],False,[],False
4,4,4,4,8.0,5,9555635,NaN,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,<p>I used Enthought's python distribution as a...,6585.0,[],"['numpy', 'scipy', 'sci-kits']",[],[],False,[],False
5,5,5,5,2.0,2,9588331,NaN,1,21,<python><pandas>,Simple cross-tabulation in pandas,"<p>I stumbled across <a href=""http://pandas.py...",9493.0,"['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['pivot/crosstab/indexing', 'Panel', 'DataFram...","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...",False,"['import pandas as pd\nAB,100.00\nAB,200.00\nA...",False
6,6,6,6,3.0,0,9652832,NaN,1,38,<python><pandas><tsv>,How to I load a tsv file into a Pandas DataFrame?,<p>I'm new to python and pandas. I'm trying t...,32040.0,['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...,"['tsv', 'DataFrame', '>>> df1 = DataFrame(csv....",['\n'],['\n'],False,['import pandas as pd\n\n'],False
7,7,7,7,6.0,0,9758450,NaN,1,34,<python><pandas>,Pandas convert dataframe to array of tuples,<p>I have manipulated some data using pandas a...,36433.0,['In [182]: data_set\nOut[182]: \n index data...,['In [182]: data_set\nOut[182]: \n index data...,"['data_set\n', '(datetime.date(2012,2,16),25.0...","['data_set\n', '(datetime.date(2012,2,16),25.0...",False,['import pandas as pd\ndata = pd.DataFrame()\n...,True
8,8,8,8,2.0,0,9762935,NaN,1,11,<python><pandas>,Add indexed column to DataFrame with pandas,"<p>I'm a beginning pandas user, and after stud...",8189.0,['df\n\n A ...,['df\n\n A ...,"['df\n\n', 'df2\n\n', '']","['df\n\n', 'df2\n\n', '']",False,['import pandas as pd\ndf = pd.DataFrame()\ndf...,True
9,9,9,9,2.0,0,9787853,NaN,1,17,<python><pandas>,join or merge with overwrite in pandas,<p>I want to perform a join/merge/append opera...,6054.0,[],"['df1', 'df2', 'df2', 'df2', 'df1', 'df1', 'df2']",[],[],False,[],False


In [2]:

import sys
sys.version

'3.5.2 (default, Nov 23 2017, 16:37:01) \n[GCC 5.4.0 20160609]'

In [3]:
# answers = answers[:50]
blackList = [22084742, 45695288, 45885112]

def filterBlackList(answers):
#     print('Before %d' % len(answers))
    answers = answers[~answers.Id.isin(blackList)]
#     print('Filtered %d' % len(answers))
    return answers

In [4]:
questions = dataset[dataset.PostTypeId == QUESTION_TYPE]

def reportCompilability(post, col):
    strcodes = post[col]
    codes = ast.literal_eval(strcodes)
    parseableCount = 0
    total = len(codes)
    errors = []
    errors_type = []
    for code in codes:
        # Preprocess
        #  code = preprocessCode(code)
        # Compile
        compiled, error, errtype = compileCode(code)
        # Update the report
        errors.append(str(error))
        errors_type.append(errtype)
        if compiled:
            parseableCount += 1
    return parseableCount, total, errors, errors_type


def compileCode(code):
    try:
#         print('Code to compile\n' + code)
        tree = ast.parse(code)
        exec(compile(tree, filename="<ast>", mode="exec"), {})
        return True, "Sucess", "Sucess"
    except Exception as e:
        return False, e, e.__class__.__name__
    
    
def compileQuestion(question, col, pre):
    # Get Answers
    answers = dataset[dataset.ParentId == question.Id]
    answers = filterBlackList(answers)
    
    print('----------------------------------------')
    print(question.Id)
    
    count = 0
    total = 0
    question_errors = []
    question_errors_type = []
    for _, answer in answers.iterrows():
        answerCompiled, answerTotal, errors, errors_type = reportCompilability(answer, col)
        print('Checking the answer Id = %s | Errors = %s' % (answer.Id, errors))
        count += answerCompiled
        total += answerTotal
        question_errors.extend(errors)
        question_errors_type.extend(errors_type)
        
    question[pre + 'Compiled'] = count
    question[pre + 'Total'] = total
    question[pre + 'Errors'] = question_errors
    question[pre + 'Errors Type'] = question_errors_type
    print('----------------------------------------')
    return question
    
analyzedQuestions = questions.apply(compileQuestion, axis=1, args=('ProcessedCode','Original',)) 


analyzedQuestions[:10]

----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7837722
Checking the answer Id = 7837947 | Errors = ["name 'close' is not defined", 'Sucess']
Checking the answer Id = 7849789 | Errors = ['Sucess', 'Sucess']
Checking the answer Id = 11617194 | Errors = ['Sucess']
Checking the answer Id = 11706782 | Errors = []
Checking the answer Id = 19415186 | Errors = ['Sucess']
Checking the answer Id = 34311080 | Errors = ["name 'pd' is not defined", 'Sucess']
----------------------------------------
----------------------------------------
8916302
Checking the answer Id = 8916746 | Errors = ["name 'df' is not defined", "name 'df' is not defined"]
----------------------------------------
----------------------------------------
8991709
C

----------------------------------------
11361985
Checking the answer Id = 11362056 | Errors = ['Sucess', "name 'paramdata' is not defined", "name 'paramdata' is not defined"]
Checking the answer Id = 11366429 | Errors = []
Checking the answer Id = 13237914 | Errors = ["name 'pandas' is not defined", "name 'pandas' is not defined"]
Checking the answer Id = 16789834 | Errors = ["name 'paramdata' is not defined"]
Checking the answer Id = 19973722 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
11391969
Checking the answer Id = 11395193 | Errors = ["name 'data' is not defined"]
Checking the answer Id = 11397052 | Errors = ["name 'data' is not defined"]
Checking the answer Id = 32783825 | Errors = ["name 'data' is not defined"]
----------------------------------------
----------------------------------------
11415701
Checking the answer Id = 11415882 | Errors = ["name 'pandas' is not defined"]
----------------------------------------


Checking the answer Id = 12555491 | Errors = ["name 'np' is not defined", "name 'df1' is not defined"]
Checking the answer Id = 12555510 | Errors = ["name 'Series' is not defined", 'Sucess', 'Sucess', "name 'df1' is not defined"]
Checking the answer Id = 13843741 | Errors = []
Checking the answer Id = 30777185 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 35387129 | Errors = ["name 'e' is not defined", "name 'df1' is not defined", "name 'pd' is not defined", "name 'np' is not defined"]
Checking the answer Id = 38510820 | Errors = []
Checking the answer Id = 43180437 | Errors = ["name 'df' is not defined", "name 'df' is not defined", "name 'pd' is not defined", "name 'pd' is not defined", "name 'pd' is not defined", "name 'pd' is not defined", "name 'pd' is not defined", "name 'protected_series' is not defined", "name 'df' is not defined", "name 'df' is not defined", 'Sucess']
----------------------------------------
----------------------------------------
12569730
C

<string>:12: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement


Checking the answer Id = 13270110 | Errors = ['Sucess']
Checking the answer Id = 32700453 | Errors = ["name 'pd' is not defined"]
----------------------------------------
----------------------------------------
13293810
Checking the answer Id = 13384494 | Errors = ["name 'read_csv' is not defined"]
----------------------------------------
----------------------------------------
13295735
Checking the answer Id = 13295801 | Errors = ["name 'df' is not defined", "name 'df' is not defined"]
Checking the answer Id = 30587837 | Errors = ["name 'pd' is not defined", "name 'df' is not defined"]
Checking the answer Id = 39478896 | Errors = ["File b'somefile.txt' does not exist"]
Checking the answer Id = 39891994 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
13331518
Checking the answer Id = 13332682 | Errors = ["name 'p' is not defined", 'Sucess']
Checking the answer Id = 13456432 | Errors = ["name 'test' is not defin

...

Checking the answer Id = 29461151 | Errors = ["name 'yourdf' is not defined", 'Sucess', "No module named 'win32com'"]
----------------------------------------
----------------------------------------
29464234
Checking the answer Id = 29464365 | Errors = ["name 'pd' is not defined", 'Sucess', 'Sucess', 'Sucess', "name 'pd' is not defined", "name 'pd' is not defined", "name 'df1' is not defined", 'Sucess', 'Sucess', 'Sucess', 'Sucess', 'Sucess', "name 'df12' is not defined"]
----------------------------------------
----------------------------------------
29498652
Checking the answer Id = 29499109 | Errors = ["name 'df' is not defined", "No module named 'matplotlib'"]
----------------------------------------
----------------------------------------
29516084
Checking the answer Id = 31026736 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
29517072
Checking the answer Id = 29517089 | Errors = ["name 'df' is not defined"]
Checking the 

Checking the answer Id = 30926717 | Errors = ["name 'pd' is not defined"]
Checking the answer Id = 30943503 | Errors = ["name 'pd' is not defined", "name 'df' is not defined", "name 'df' is not defined"]
----------------------------------------
----------------------------------------
31029560
Checking the answer Id = 31029857 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 31029861 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 31033603 | Errors = ["No module named 'statsmodels'"]
----------------------------------------
----------------------------------------
31037298
Checking the answer Id = 31037360 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
31247198
Checking the answer Id = 31247247 | Errors = ["name 'np' is not defined", 'Sucess', "name 'df' is not defined"]
Checking the answer Id = 31247279 | Errors = ['Sucess']
----------------------------------------
-------------

----------------------------------------
33530753
----------------------------------------
----------------------------------------
33768122
Checking the answer Id = 33777664 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 33780558 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
33813815
Checking the answer Id = 33814054 | Errors = []
----------------------------------------
----------------------------------------
33869292
Checking the answer Id = 33873209 | Errors = ["No module named 'bokeh'"]
----------------------------------------
----------------------------------------
33952142
Checking the answer Id = 33952294 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
33957720
Checking the answer Id = 33957850 | Errors = ["name 'df' is not defined", "name 'df' is not defined"]
----------------------------------------
---------

Checking the answer Id = 39363715 | Errors = ["name 'pd' is not defined", "No module named 'seaborn'", "name 'np' is not defined", "No module named 'bokeh'"]
----------------------------------------
----------------------------------------
39321914
----------------------------------------
----------------------------------------
39674713
Checking the answer Id = 40005797 | Errors = ['Sucess', 'Sucess', "name 'df' is not defined", "name 'df' is not defined", "No module named 'keras'", "name 'np' is not defined", 'Sucess', "name 'X_train' is not defined", "No module named 'keras'", 'Sucess', 'Sucess']
----------------------------------------
----------------------------------------
40143902
----------------------------------------
----------------------------------------
40144769
Checking the answer Id = 40145561 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
40209520
Checking the answer Id = 40209737 | Errors = [

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,AnswerCount,CommentCount,Id,ParentId,PostTypeId,Score,Tags,Title,Body,ViewCount,Code,AllCode,ProcessedCode,PandasImportCode,PandasImportNormalized,DataFrameCode,DataFrameNormalized,OriginalCompiled,OriginalTotal,OriginalErrors,OriginalErrors Type
0,0,0,0,$2.00,7,7776679,$nan,1,25,<python><pandas>,append two data frame with pandas,<p>When I try to merge two dataframes by rows ...,"$25,479.00","['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', '', '', '']","['bigdata = data1.append(data2)\n', '', '', '']",False,['import pandas as pd\ndata2 = pd.DataFrame()\...,True,0,0,[],[]
1,1,1,1,$7.00,3,7837722,$nan,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,<p>I want to perform my own complex operations...,"$195,364.00","['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['2011-10-19,27.37,27.47,27.01,27.13,42880000,...","['2011-10-19,27.37,27.47,27.01,27.13,42880000,...",False,"['import pandas as pd\n2011-10-19,27.37,27.47,...",False,6,8,"[name 'close' is not defined, Sucess, Sucess, ...","[NameError, Sucess, Sucess, Sucess, Sucess, Su..."
2,2,2,2,$2.00,0,8916302,$nan,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,<p>I have a dataframe <code>df</code> in panda...,"$19,805.00","['df_greater_than10 = df[df[""colA""] > 10]\n']","['df', 'pandas.read_table', 'colA', 'df_greate...","['df_greater_than10 = df[df[""colA""] > 10]\n']","['df_greater_than10 = df[df[""colA""] > 10]\n']",False,['import pandas as pd\ndf = pd.DataFrame()\ndf...,True,0,2,"[name 'df' is not defined, name 'df' is not de...","[NameError, NameError]"
3,3,3,3,$3.00,16,8991709,$nan,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,"<p>I recently came across the <a href=""http://...","$16,173.00",[],"['pandas', 'data.table', 'data.table', 'merge(...",[],[],False,[],False,0,0,[],[]
4,4,4,4,$8.00,5,9555635,$nan,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,<p>I used Enthought's python distribution as a...,"$6,585.00",[],"['numpy', 'scipy', 'sci-kits']",[],[],False,[],False,0,0,[],[]
5,5,5,5,$2.00,2,9588331,$nan,1,21,<python><pandas>,Simple cross-tabulation in pandas,"<p>I stumbled across <a href=""http://pandas.py...","$9,493.00","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['pivot/crosstab/indexing', 'Panel', 'DataFram...","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...",False,"['import pandas as pd\nAB,100.00\nAB,200.00\nA...",False,0,5,"[name 'category' is not defined, File b'/path/...","[NameError, FileNotFoundError, NameError, Name..."
6,6,6,6,$3.00,0,9652832,$nan,1,38,<python><pandas><tsv>,How to I load a tsv file into a Pandas DataFrame?,<p>I'm new to python and pandas. I'm trying t...,"$32,040.00",['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...,"['tsv', 'DataFrame', '>>> df1 = DataFrame(csv....",['\n'],['\n'],False,['import pandas as pd\n\n'],False,0,2,"[name 'DataFrame' is not defined, name 'DataFr...","[NameError, NameError]"
7,7,7,7,$6.00,0,9758450,$nan,1,34,<python><pandas>,Pandas convert dataframe to array of tuples,<p>I have manipulated some data using pandas a...,"$36,433.00",['In [182]: data_set\nOut[182]: \n index data...,['In [182]: data_set\nOut[182]: \n index data...,"['data_set\n', '(datetime.date(2012,2,16),25.0...","['data_set\n', '(datetime.date(2012,2,16),25.0...",False,['import pandas as pd\ndata = pd.DataFrame()\n...,True,0,3,"[name 'data_set' is not defined, name 'data_se...","[NameError, NameError, NameError]"
8,8,8,8,$2.00,0,9762935,$nan,1,11,<python><pandas>,Add indexed column to DataFrame with pandas,"<p>I'm a beginning pandas user, and after stud...","$8,189.00",['df\n\n A ...,['df\n\n A ...,"['df\n\n', 'df2\n\n', '']"

In [5]:
analyzedQuestions = analyzedQuestions.apply(compileQuestion, axis=1, args=('PandasImportCode','PandasImport',)) 

----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7837722
Checking the answer Id = 7837947 | Errors = ["name 'close' is not defined", 'Sucess']
Checking the answer Id = 7849789 | Errors = ['Sucess', 'Sucess']
Checking the answer Id = 11617194 | Errors = ['Sucess']
Checking the answer Id = 11706782 | Errors = []
Checking the answer Id = 19415186 | Errors = ['Sucess']
Checking the answer Id = 34311080 | Errors = ["name 'time' is not defined", 'Sucess']
----------------------------------------
----------------------------------------
8916302
Checking the answer Id = 8916746 | Errors = ["name 'df' is not defined", "name 'df' is not defined"]
----------------------------------------
----------------------------------------
8991709

Checking the answer Id = 11077060 | Errors = []
Checking the answer Id = 11077215 | Errors = []
----------------------------------------
----------------------------------------
11106823
Checking the answer Id = 11107627 | Errors = ["name 'x' is not defined", 'Sucess']
Checking the answer Id = 11112419 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
11232275
Checking the answer Id = 13833239 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 19600533 | Errors = ["name 'pandas' is not defined"]
----------------------------------------
----------------------------------------
11285613
Checking the answer Id = 11287278 | Errors = ["name 'df' is not defined", "name 'df' is not defined", "name 'df' is not defined"]
Checking the answer Id = 11385335 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 13165753 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 25643178 | Errors = ["name 'df' is no

Checking the answer Id = 12056933 | Errors = ["No module named 'pydobc'"]
Checking the answer Id = 12060886 | Errors = ["name 'resoverall' is not defined"]
Checking the answer Id = 14487936 | Errors = ["name 'sql' is not defined", "No module named 'pyodbc'"]
Checking the answer Id = 19031661 | Errors = ["name 'query' is not defined"]
----------------------------------------
----------------------------------------
12052067
Checking the answer Id = 12077782 | Errors = []
Checking the answer Id = 12184679 | Errors = []
----------------------------------------
----------------------------------------
12065885
Checking the answer Id = 12065904 | Errors = []
Checking the answer Id = 19295726 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 26724725 | Errors = ['Sucess', 'Sucess', 'Sucess', "type object 'str' has no attribute 'contains'"]
Checking the answer Id = 29108799 | Errors = ["name 'rpt' is not defined", "name 'rpt' is not defined"]
-----------------------------------

Checking the answer Id = 32152755 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
13019719
Checking the answer Id = 13020027 | Errors = ["name 's' is not defined", "name 'datetime' is not defined", "name 's' is not defined"]
Checking the answer Id = 31907749 | Errors = ["name 'BDay' is not defined", 'Sucess']
Checking the answer Id = 33011646 | Errors = []
Checking the answer Id = 33480745 | Errors = ['Non-vectorized DateOffset being applied to Series or DatetimeIndex', 'Sucess']
----------------------------------------
----------------------------------------
13021654
Checking the answer Id = 13021797 | Errors = ['Sucess']
Checking the answer Id = 13036844 | Errors = []
----------------------------------------
----------------------------------------
13035764
Checking the answer Id = 13036848 | Errors = ["name 'df3' is not defined", "'select' is deprecated and will be removed in a future release. You can use .lo

Checking the answer Id = 13485766 | Errors = []
Checking the answer Id = 18145399 | Errors = ["name 'df' is not defined", "name 'df' is not defined", "name 'df' is not defined"]
Checking the answer Id = 22596982 | Errors = ['Sucess']
Checking the answer Id = 31431997 | Errors = ["name 'df' is not defined", "name 'df' is not defined", "name 'df' is not defined", 'Sucess', 'Sucess', 'Sucess', 'Sucess']
Checking the answer Id = 34576537 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 35385805 | Errors = ['Sucess']
Checking the answer Id = 36958937 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 37000877 | Errors = []
Checking the answer Id = 45357725 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
13413590
Checking the answer Id = 13413845 | Errors = ["name 'df' is not defined"]
Checking the answer Id = 13434501 | Errors = ["name 'np' is not defined", "name 'df' is not defined", "na

...

----------------------------------------
30357276
Checking the answer Id = 30357382 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
30462807
Checking the answer Id = 30470630 | Errors = []
----------------------------------------
----------------------------------------
30519487
Checking the answer Id = 31257931 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
30522724
Checking the answer Id = 30522778 | Errors = ["name 'percentile_list' is not defined", 'Sucess']
----------------------------------------
----------------------------------------
30522982
Checking the answer Id = 30523225 | Errors = []
Checking the answer Id = 30525128 | Errors = []
----------------------------------------
----------------------------------------
30530663
Checking the answer Id = 30531939 | Errors = ['Sucess']
----------------------------------------
---------------------------

Checking the answer Id = 39186403 | Errors = ["name 'df' is not defined", 'Sucess']
----------------------------------------
----------------------------------------
32093829
Checking the answer Id = 32094352 | Errors = ["name 'df' is not defined", "name 'df' is not defined", 'Sucess']
----------------------------------------
----------------------------------------
32244019
Checking the answer Id = 32244161 | Errors = ["No module named 'matplotlib'"]
----------------------------------------
----------------------------------------
32244753
Checking the answer Id = 32245025 | Errors = ["name 'sns_plot' is not defined", "name 'sns_plot' is not defined"]
Checking the answer Id = 32245026 | Errors = ["name 'sns' is not defined"]
Checking the answer Id = 39482402 | Errors = ['Sucess', "name 'sns' is not defined", "name 'myGridPlotObject' is not defined"]
Checking the answer Id = 41452422 | Errors = ["name 'sns_plot' is not defined"]
----------------------------------------
----------------

----------------------------------------
----------------------------------------
32888124
Checking the answer Id = 37226672 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
32957441
Checking the answer Id = 33004253 | Errors = ['Sucess', "name 'df' is not defined", "name 'df' is not defined"]
----------------------------------------
----------------------------------------
33086881
Checking the answer Id = 33086953 | Errors = ["name 'df1' is not defined"]
----------------------------------------
----------------------------------------
33088010
Checking the answer Id = 33088410 | Errors = ["name 'df_a' is not defined"]
----------------------------------------
----------------------------------------
33109107
Checking the answer Id = 33109272 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
33126477
----------------------------------------
--------------------------------------

----------------------------------------
----------------------------------------
36539396
Checking the answer Id = 36539513 | Errors = ['Sucess', 'Sucess']
----------------------------------------
----------------------------------------
36631163
Checking the answer Id = 38650886 | Errors = []
----------------------------------------
----------------------------------------
36684013
Checking the answer Id = 36685531 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
36694313
Checking the answer Id = 36694513 | Errors = ["module 'pandas.core' has no attribute 'format'", "module 'pandas.core' has no attribute 'format'", "module 'pandas.core' has no attribute 'format'", "module 'pandas' has no attribute 'formats'"]
----------------------------------------
----------------------------------------
36891977
Checking the answer Id = 36893675 | Errors = ["name 'df1' is not defined"]
----------------------------------------

Checking the answer Id = 44724677 | Errors = ["name 'df1' is not defined", "name 'df1' is not defined", 'Sucess', "name 'concat' is not defined", 'No columns to parse from file']
Checking the answer Id = 44800142 | Errors = ["name 'df1' is not defined"]
----------------------------------------
----------------------------------------
44946141
Checking the answer Id = 44948390 | Errors = ["File b'./data.csv' does not exist"]
Checking the answer Id = 45013739 | Errors = ["name 'df' is not defined"]
----------------------------------------
----------------------------------------
44998223
Checking the answer Id = 44998387 | Errors = ['Sucess', "name 'df' is not defined"]
Checking the answer Id = 44999009 | Errors = ['Sucess', "name 'df' is not defined", 'Sucess', "name 'df2' is not defined", 'Sucess', 'Sucess', 'Sucess', 'Sucess']
----------------------------------------
----------------------------------------
45193131
Checking the answer Id = 45193340 | Errors = ["name 'df' is not defin

In [6]:
analyzedQuestions = analyzedQuestions.apply(compileQuestion, axis=1, args=('DataFrameCode','DataFrame',)) 


----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7776679
Checking the answer Id = 7779260 | Errors = []
----------------------------------------
----------------------------------------
7837722
Checking the answer Id = 7837947 | Errors = ["name 'close' is not defined", 'Sucess']
Checking the answer Id = 7849789 | Errors = ['Sucess', 'Sucess']
Checking the answer Id = 11617194 | Errors = ['Sucess']
Checking the answer Id = 11706782 | Errors = []
Checking the answer Id = 19415186 | Errors = ['Sucess']
Checking the answer Id = 34311080 | Errors = ["name 'time' is not defined", 'Sucess']
----------------------------------------
----------------------------------------
8916302
Checking the answer Id = 8916746 | Errors = ['Sucess', "'A'"]
----------------------------------------
----------------------------------------
8991709
Checking the answer Id = 8992714 | Err

Checking the answer Id = 11397052 | Errors = ["'date'"]
Checking the answer Id = 32783825 | Errors = ["'DataFrame' object has no attribute 'date'"]
----------------------------------------
----------------------------------------
11415701
Checking the answer Id = 11415882 | Errors = ["name 'pandas' is not defined"]
----------------------------------------
----------------------------------------
11418192
Checking the answer Id = 11420594 | Errors = ['Sucess', "name 'np' is not defined", "name 'mask' is not defined"]
Checking the answer Id = 11475486 | Errors = ["name 'pandas' is not defined"]
----------------------------------------
----------------------------------------
11495051
Checking the answer Id = 11495086 | Errors = ['Sucess', 'Sucess']
----------------------------------------
----------------------------------------
11548005
Checking the answer Id = 11548224 | Errors = []
----------------------------------------
----------------------------------------
11587782
Checking the 

----------------------------------------
----------------------------------------
12286607
Checking the answer Id = 12286958 | Errors = ["No module named 'matplotlib'"]
Checking the answer Id = 29528483 | Errors = ["No module named 'seaborn'"]
----------------------------------------
----------------------------------------
12307099
Checking the answer Id = 12307162 | Errors = ["name 'np' is not defined", '\n.ix is deprecated. Please use\n.loc for label based indexing or\n.iloc for positional indexing\n\nSee the documentation here:\nhttp://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated']
Checking the answer Id = 12607018 | Errors = ['Sucess']
Checking the answer Id = 44913631 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
12322289
Checking the answer Id = 12326113 | Errors = ['Sucess']
Checking the answer Id = 12336039 | Errors = ["name 'quotes' is not defined"]
---------------------------------------

Checking the answer Id = 12882439 | Errors = ['Sucess', 'Sucess', "name 'Bob' is not defined"]
----------------------------------------
----------------------------------------
12945971
Checking the answer Id = 13674286 | Errors = ["No module named 'matplotlib'"]
----------------------------------------
----------------------------------------
12960574
Checking the answer Id = 12961158 | Errors = []
----------------------------------------
----------------------------------------
13003051
Checking the answer Id = 13003524 | Errors = ['\n.ix is deprecated. Please use\n.loc for label based indexing or\n.iloc for positional indexing\n\nSee the documentation here:\nhttp://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated']
Checking the answer Id = 32152755 | Errors = ['\n.ix is deprecated. Please use\n.loc for label based indexing or\n.iloc for positional indexing\n\nSee the documentation here:\nhttp://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-

----------------------------------------
13575090
Checking the answer Id = 13581730 | Errors = ['Sucess', 'No objects to concatenate']
Checking the answer Id = 26803731 | Errors = ["name 'd' is not defined", "name 'd' is not defined", "name 'd' is not defined", "name 'd' is not defined", "name 'd' is not defined"]
----------------------------------------
----------------------------------------
13582449
Checking the answer Id = 13583024 | Errors = ["name 'A' is not defined"]
----------------------------------------
----------------------------------------
13611065
Checking the answer Id = 13616382 | Errors = ["'col1'", 'Sucess', "name 'col1' is not defined"]
Checking the answer Id = 30778300 | Errors = ["'DataFrame' object has no attribute 'col1'"]
----------------------------------------
----------------------------------------
13636592
Checking the answer Id = 13866073 | Errors = ["name 'songs' is not defined"]
Checking the answer Id = 19473752 | Errors = ["'DataFrame' object has no 

...

Checking the answer Id = 29528804 | Errors = ["name 'query' is not defined"]
Checking the answer Id = 36257640 | Errors = ["name 'session' is not defined"]
----------------------------------------
----------------------------------------
29530232
Checking the answer Id = 29530303 | Errors = []
Checking the answer Id = 29530559 | Errors = ['Sucess', 'Sucess', 'Sucess', 'Sucess', 'Sucess', 'Sucess', 'Sucess']
Checking the answer Id = 29530601 | Errors = ['Sucess', 'Sucess']
Checking the answer Id = 37442692 | Errors = ['Sucess', 'Sucess']
----------------------------------------
----------------------------------------
29532894
Checking the answer Id = 29533502 | Errors = ["No module named 'matplotlib'"]
Checking the answer Id = 29533687 | Errors = ["No module named 'matplotlib'"]
----------------------------------------
----------------------------------------
29545704
Checking the answer Id = 29546836 | Errors = ['Sucess', 'Sucess']
Checking the answer Id = 29548349 | Errors = ['Sucess

----------------------------------------
31037298
Checking the answer Id = 31037360 | Errors = ["'weight'"]
----------------------------------------
----------------------------------------
31247198
Checking the answer Id = 31247247 | Errors = ["name 'np' is not defined", 'Sucess', 'Sucess']
Checking the answer Id = 31247279 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
31329627
Checking the answer Id = 31331100 | Errors = []
Checking the answer Id = 41150420 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
31331358
Checking the answer Id = 31331449 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
31357611
Checking the answer Id = 31357733 | Errors = ["name '_converter' is not defined"]
Checking the answer Id = 35446404 | Errors = ["No module named 'matplotlib'"]
Checking the answer Id = 36319915 | Errors = ["No module n

Checking the answer Id = 41880513 | Errors = ['Sucess']
----------------------------------------
----------------------------------------
32444138
Checking the answer Id = 32444187 | Errors = ["name 'list_of_dataframes' is not defined"]
----------------------------------------
----------------------------------------
32468402
Checking the answer Id = 32469151 | Errors = ['Sucess']
Checking the answer Id = 32470490 | Errors = ["'name'", 'Sucess']
----------------------------------------
----------------------------------------
32565302
Checking the answer Id = 32583988 | Errors = []
Checking the answer Id = 40679977 | Errors = []
----------------------------------------
----------------------------------------
32591466
Checking the answer Id = 32591786 | Errors = ['Sucess', 'The `sheetname` keyword is deprecated, use `sheet_name` instead']
Checking the answer Id = 32591799 | Errors = ["name 'pandas' is not defined"]
----------------------------------------
------------------------------

----------------------------------------
----------------------------------------
35414625
Checking the answer Id = 35415751 | Errors = ['Sucess', 'Sucess']
Checking the answer Id = 35416021 | Errors = ["'year'", 'Sucess']
----------------------------------------
----------------------------------------
35634238
Checking the answer Id = 35715029 | Errors = ["No module named 'matplotlib'", 'Sucess', "name 'df' is not defined", '\n.ix is deprecated. Please use\n.loc for label based indexing or\n.iloc for positional indexing\n\nSee the documentation here:\nhttp://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated', 'cannot copy sequence with size 2 to array axis with dimension 0', "name 'table' is not defined"]
----------------------------------------
----------------------------------------
35782929
----------------------------------------
----------------------------------------
35826912
----------------------------------------
----------------------------------

Checking the answer Id = 40005797 | Errors = ['Sucess', 'Sucess', "name 'input_cols' is not defined", "name 'output_cols' is not defined", "No module named 'keras'", "name 'np' is not defined", 'Sucess', "name 'X_train' is not defined", "No module named 'keras'", 'Sucess', 'Sucess']
----------------------------------------
----------------------------------------
40143902
----------------------------------------
----------------------------------------
40144769
Checking the answer Id = 40145561 | Errors = ['single positional indexer is out-of-bounds']
----------------------------------------
----------------------------------------
40209520
Checking the answer Id = 40209737 | Errors = ['Sucess', "Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'"]
----------------------------------------
----------------------------------------
40225683
Empty DataFrame
Columns: []
Index: []
Checking the answer Id = 40225796 | Errors = ['Sucess']
--------------

In [ ]:
# pd.set_option('display.max_colwidth', -1)
# analyzedQuestions[['Id', 'Title', 'Code', 'AllCode', 'Compiled','Total',  'Errors']]
# analyzedQuestions[:10]

In [7]:
analyzedQuestions['OriginalCompiled'].sum()
analyzedQuestions['OriginalTotal'].sum()

analyzedQuestions['PandasImportCompiled'].sum()
analyzedQuestions['PandasImportTotal'].sum()

analyzedQuestions['DataFrameCompiled'].sum()
analyzedQuestions['DataFrameTotal'].sum()



1142

3659

1254

3659

1551

3659

In [ ]:
def report_errors(analyzedQuestions, prefix):
    
    errorslist = []

    for err in analyzedQuestions[prefix + 'Errors Type']:
        errorslist.extend(err)

    df = pd.DataFrame(errorslist)   
    print(df[0].value_counts())
    
report_errors(analyzedQuestions, 'Original')
report_errors(analyzedQuestions, 'PandasImport')
report_errors(analyzedQuestions, 'DataFrame')

In [ ]:
analyzedQuestions.drop(columns=['Body']).to_csv(os.path.join('..', 'data', 'pandas_compiled_code.csv'), sep=';', doublequote=True)